In [1]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# get dataframes
df_pid = pd.read_csv('./hospiedata/data/cleaned/id_to_attrs_no_multi_ehr.csv')
df_care = pd.read_csv('./hospiedata/data/care_translated_all_cols.csv', parse_dates=['TimeStamp_Insert'])
df_ht = pd.read_csv('./hospiedata/data/cleaned/therapy_cleaned.csv')

C:\Users\prate\Anaconda3\envs\det2\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (7,11,13,15,16,17,18,19,20,22,23,24,25,26,27,33,34) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\prate\Anaconda3\envs\det2\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# fix dtype errors
# df_pid.dtypes

In [4]:
# df_care.dtypes
df_care.columns

Index(['Unnamed: 0', 'IDEHR', 'Author_Insert', 'TimeStamp_Insert', 'IDAccess',
       'EHRType', 'PatientID', 'Note_Non_Rilevabile', 'IDGESTIONE_AUSILI',
       'ds_ncons', 'ds_nbolla', 'dt_consegna', 'ds_nritiro', 'dt_ritiro',
       'opt_annulla_consegna', 'ds_note_x', 'dt_Ric_consegna',
       'dt_ric_cons_forn', 'dt_ric_ritiro', 'dt_ric_ritiro_forn',
       'opt_ausilio', 'IDTERAPIE_OUTPAT_VIDAS', 'ds_altro_farmaco', 'ds_dose',
       'opt_via_di_somm', 'ds_ora', 'dt_data_inizio', 'ds_note_y',
       'opt_pregressa', 'opt_somm_terapia', 'opt_estemporanea', 'opt_termina',
       'opt_somm_in_pompa', 'opt_farmaco', 'Note_al_bisogno'],
      dtype='object')

In [5]:
# df_ht.dtypes

In [6]:
# split the dframe into considering only those IDEHR whose values appear in both 
print(len(df_pid.idehr.unique()))
print(len(df_care.IDEHR.unique()))
pid_set = set(df_pid.idehr.unique())
care_set = set(df_care.IDEHR.unique())
common_set = list(care_set.intersection(pid_set))
print(len(common_set))

6790
4300
4077


In [7]:
# get the new care dframe
print(df_care.shape)
df_care = df_care[df_care['IDEHR'].isin(common_set)]
print(df_care.shape)

(105519, 35)
(102404, 35)


In [8]:
df_care.head(2)

,Unnamed: 0,IDEHR,Author_Insert,TimeStamp_Insert,IDAccess,EHRType,PatientID,Note_Non_Rilevabile,IDGESTIONE_AUSILI,ds_ncons,...,ds_ora,dt_data_inizio,ds_note_y,opt_pregressa,opt_somm_terapia,opt_estemporanea,opt_termina,opt_somm_in_pompa,opt_farmaco,Note_al_bisogno
18,18,203.0,visconti giovanna - vscgnn70t70f205e,2015-06-15 13:19:04,9860.0,amb,73.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,20,209.0,visconti giovanna - vscgnn70t70f205e,2015-06-15 13:37:20,9864.0,amb,75.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df_care.opt_farmaco.value_counts()

other (see notes) # 2004                                                                                      9018
furosemide (25 mg lasix tablets) # 1223                                                                       2286
dexamethasone (soldesam os gtt 0-2% gtt) # 1446                                                               1938
morphine hydrochloride (10 mg morphine hydrochloride fl) # 1598                                               1723
scopolamine butylbromide (buscopan 20mg / ml fl) # 997                                                        1549
                                                                                                              ... 
timolol (0.25% timoptol coll) # 1974                                                                             1
timolol (droptimol 0-25% coll) # 1972                                                                            1
venlafaxine (150 mg efexor cps rp) # 1918 = 0                                   

In [10]:
# create the new columns
df_care['DrugName'] = ''
df_care['Dose'] = np.nan
df_care['DrugRef'] = np.nan
df_care['DrugNote'] = ''

In [11]:
df_care.head(2)

,Unnamed: 0,IDEHR,Author_Insert,TimeStamp_Insert,IDAccess,EHRType,PatientID,Note_Non_Rilevabile,IDGESTIONE_AUSILI,ds_ncons,...,opt_somm_terapia,opt_estemporanea,opt_termina,opt_somm_in_pompa,opt_farmaco,Note_al_bisogno,DrugName,Dose,DrugRef,DrugNote
18,18,203.0,visconti giovanna - vscgnn70t70f205e,2015-06-15 13:19:04,9860.0,amb,73.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,
20,20,209.0,visconti giovanna - vscgnn70t70f205e,2015-06-15 13:37:20,9864.0,amb,75.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,


In [12]:
# get the drug/diagnosis reference number
df_care['DrugRef'] = df_care.opt_farmaco.str.findall('(?<=#.).[0-9]+').to_frame()
df_care['DrugRef'] = df_care['DrugRef'].apply(lambda x: x if isinstance(x, float) else float(x[0]))
print(df_care['DrugRef'][105517])
# print(df_care.DrugRef.unique())

1626.0


In [13]:
# get the drug/active ingredient name
df_care['DrugName'] = df_care.opt_farmaco.str.findall('(.+).\(').to_frame()
df_care['DrugName'] = df_care['DrugName'].apply(lambda x: x if isinstance(x, float) else x[0])
print(df_care['DrugName'][105517])

oxycodone / acetaminophen


In [14]:
df_care['opt_farmaco'][105517]

'oxycodone / acetaminophen (depalgos 10 + 325 mg tablets rev) # 1626'

In [15]:
# capture everything within the brackets
df_care['DrugNote'] = df_care.opt_farmaco.str.findall('\((.+)\)').to_frame()
df_care['DrugNote'] = df_care['DrugNote'].apply(lambda x: x if isinstance(x, float) else x[0])
print(df_care['DrugNote'][105517])

depalgos 10 + 325 mg tablets rev


In [41]:
df_care.head(10)

,Unnamed: 0,IDEHR,Author_Insert,TimeStamp_Insert,IDAccess,EHRType,PatientID,Note_Non_Rilevabile,IDGESTIONE_AUSILI,ds_ncons,...,opt_somm_terapia,opt_estemporanea,opt_termina,opt_somm_in_pompa,opt_farmaco,Note_al_bisogno,DrugName,Dose,DrugRef,DrugNote
18,18,203.0,visconti giovanna - vscgnn70t70f205e,2015-06-15 13:19:04,9860.0,amb,73.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,20,209.0,visconti giovanna - vscgnn70t70f205e,2015-06-15 13:37:20,9864.0,amb,75.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,25,209.0,visconti giovanna - vscgnn70t70f205e,2015-06-22 10:35:58,9928.0,amb,75.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,26,203.0,visconti giovanna - vscgnn70t70f205e,2015-06-22 10:38:14,9929.0,amb,73.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,29,2536.0,visconti giovanna - vscgnn70t70f205e,2015-06-25 12:20:07,9977.0,amb,223.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
31,31,209.0,visconti giovanna - vscgnn70t70f205e,2015-06-29 12:28:47,10016.0,amb,75.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33,33,203.0,visconti giovanna - vscgnn70t70f205e,2015-06-29 12:32:43,10018.0,amb,73.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
34,34,2536.0,visconti giovanna - vscgnn70t70f205e,2015-07-02 12:05:59,10062.0,amb,223.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,40,209.0,visconti giovanna - vscgnn70t70f205e,2015-07-06 12:47:49,10097.0,amb,75.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,41,203.0,visconti giovanna - vscgnn70t70f205e,2015-07-06 12:49:17,10098.0,amb,73.0,nr,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
# create new dframe purely for the drugs used
df_drug = df_care[['DrugName', 'Dose', 'DrugRef', 'DrugNote']].drop_duplicates().sort_values(by=['DrugName'], ascending=True).reset_index()
# split the string by '+'
df_drug['t1:+'] = df_drug.DrugNote.str.findall('(.+)\+(.+)')
df_drug['t1:+'] = df_drug['t1:+'].apply(lambda x: x if isinstance(x, float) else x)
df_drug['Dose'] = df_drug.DrugNote.str.findall('.+\s(.+mg)')
df_drug['Dose'] = df_drug['Dose'].apply(lambda x: np.nan if isinstance(x, float) or not x else x[0])
# split the string by '/'
df_drug['t2:/'] = df_drug.DrugName.str.findall('(.+)\s/\s(.+)')
df_drug['t2:/'] = df_drug['t2:/'].apply(lambda x: x if isinstance(x, float) else x)
# split the doses for those entries containing two drugs in the same row
df_drug['t3:+d'] = df_drug.DrugNote.str.findall('(\d+)')
df_drug['t3:+d'] = df_drug['t3:+d'].apply(lambda x: x if isinstance(x, float) else x)
print(df_drug.shape)
df_drug.head(50)

(1042, 8)


,index,DrugName,Dose,DrugRef,DrugNote,t1:+,t2:/,t3:+d
0,24374,+ acetaminophen codeine sulfate,NaN,1634.0,30 mg + 500 tachidol bust,"[(30 mg , 500 tachidol bust)]",[],"[30, 500]"
1,32948,+ acetaminophen codeine sulfate,NaN,1633.0,30 mg + 500 coefferalgan bust,"[(30 mg , 500 coefferalgan bust)]",[],"[30, 500]"
2,53281,+ acetaminophen codeine sulfate,NaN,1632.0,30 mg tablets codamol 500+ rev,"[(30 mg tablets codamol 500, rev)]",[],"[30, 500]"
3,30847,acarbose,50 mg,1096.0,glucobay 50 mg tablets,[],[],[50]
4,75998,acarbose,50 mg,1093.0,acarphage 50 mg tablets,[],[],[50]
5,26800,acarbose,100 mg,1094.0,glicobase 100 mg tablets,[],[],[100]
6,24965,acarbose,100 mg,1095.0,glucobay 100 mg tablets,[],[],[100]
7,24966,acarbose,100 mg,1092.0,acarphage 100 mg tablets,[],[],[100]
8,24352,acenocoumarol,1 mg,1126.0,sintrom tablets 1 mg,[],[],[1]
9,35413,acenocoumarol,4 mg,1127.0,sintrom 4 mg tablets div,[],[],[4]


In [27]:
# test
df_drug['t2:/'][17][0]

('acetaminophen', 'codeine')

In [18]:
# examples of where problems might occur during regex
print(df_care['opt_farmaco'][23959])
print(df_care['opt_farmaco'][23973])
print(df_care['opt_farmaco'][23679])

acetazolamide (diamox * 12 tablets 250 mg - 12 tablets 250 mg diamox *) # 2130 = 0
albuterol (ventolin * susp inal 200 erog 100 mcg - ventolin * susp inal 200 erog 100 mcg) # 2119 = 0
allopurinol (teva allopurinol italy - 300 mg tablets 30 tablets in blisters pvc / pvdc / al) # 1708 = 0


In [39]:
# crate a new dframe where the drugs having different trade names but the same active ingridient and dose are combined
df_drug_long = df_care[['IDEHR', 'TimeStamp_Insert', 'EHRType', 'ds_dose', 'opt_farmaco', 'DrugName', 'Dose', 'DrugRef', 'DrugNote']].reset_index().rename(columns={"index": "df_care_index"})
# get the drug names
df_drug_long['DrugName'] = df_drug_long.DrugName.str.findall('([a-z\s]+)')
df_drug_long['DrugName'] = df_drug_long['DrugName'].apply(lambda x: x if isinstance(x,float) else x)
# get the doses
df_drug_long['Dose'] = df_drug_long.DrugNote.str.findall('(\d*\.?\d+\s[a-z]+)')
df_drug_long['Dose'] = df_drug_long['Dose'].apply(lambda x: x if isinstance(x, float) else x)
df_drug_long.tail(50)

,df_care_index,IDEHR,TimeStamp_Insert,EHRType,ds_dose,opt_farmaco,DrugName,Dose,DrugRef,DrugNote
102354,105469,12189.0,2017-07-31 10:21:54,amb,1 cp,brotizolam (lendormin 0.25 mg tablets) # 1890,[brotizolam],[0.25 mg],1890.0,lendormin 0.25 mg tablets
102355,105470,12189.0,2017-07-31 10:21:59,amb,1 cp,oxycodone (10 mg tablets oxycontin rp) # 1617,[oxycodone],[10 mg],1617.0,10 mg tablets oxycontin rp
102356,105471,9563.0,2017-07-31 10:22:07,amb,16 gtt,dexamethasone (0.2% soldesam os gtt gtt) # 144...,[dexamethasone],[],1446.0,0.2% soldesam os gtt gtt
102357,105472,12157.0,2017-07-31 10:22:34,amb,2019-01-02 00:00:00,other (see notes) # 2004,[other],[],2004.0,see notes
102358,105473,12169.0,2017-07-31 11:14:43,amb,1 tablespoon,itraconazole (triasporin 150 ml 10 mg / ml os)...,[itraconazole],"[150 ml, 10 mg]",1539.0,triasporin 150 ml 10 mg / ml os
102359,105474,12169.0,2017-07-31 11:14:50,amb,1,oxycodone / naloxone (targin 20 + 10 mg tablet...,"[oxycodone , naloxone]",[10 mg],1623.0,targin 20 + 10 mg tablets rp
102360,105475,11742.0,2017-07-31 11:43:56,amb,1 tablet,nebivolol (nebivolol 5 mg tablets) # 1270,[nebivolol],[5 mg],1270.0,nebivolol 5 mg tablets
102361,105476,11742.0,2017-07-31 11:44:05,amb,1 syringe,"lovenox® (t clexane 6,000 iu) # 1135",[lovenox],[000 iu],1135.0,"t clexane 6,000 iu"
102362,105477,11742.0,2017-07-31 11:44:11,amb,1/2 cpr,ramipril (ramipril 5 mg tablets) # 1316,[ramipril],[5 mg],1316.0,ramipril 5 mg tablets
102363,105478,11742.0,2017-07-31 11:44:16,amb,6 iu,other (see notes) # 2004,[other],[],2004.0,see notes


In [40]:
# print([x for x in df_drug_long.DrugName.unique()])